In [1]:
import sys
from common import commons
home = commons.home
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit,cross_validate
from common import Convert_To_Normal_Dist as ctnd
from features_selection import TTest as tt
from matplotlib import pyplot as plt
from features_selection import CorrFeatureSelector as cfs
from simulation import AutoencoderSimulator as AS
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from features_selection import Feature_Selection as FS
from common import DataScaler as ds
from log import Logger
from feature_engineering import Sparse_Autoencoder_class as sac
from hyperopt import fmin,tpe,hp, STATUS_OK,Trials
from sklearn.preprocessing import StandardScaler
import math
from features_selection import TTest, WilcoxonRankSums
from sklearn.externals import joblib

/home/ec2-user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#--------------------------------------------------------------------- 
def normalize_feature(all_features,dataset='AD_CpG'):
    count_thresh = 10
    unique_value_counts = [len(all_features[col].unique()) for col in all_features.columns[2:]] #ignore chr and coordinate
    normalize_all_features = all_features.copy()
    mappings = pd.DataFrame()
    for col,num in zip(all_features.columns[2:],unique_value_counts):
        if num >= count_thresh:
            print(col)
            atn = ctnd.AnyToNormal(col)
            mapping = atn.transform(normalize_all_features)
            mappings[mapping.columns[0]] = mapping.ix[:,0]
            mappings[mapping.columns[1]] = mapping.ix[:,1]
        else: 
            continue
    h5s = pd.HDFStore(home+'data/'+dataset+'/normalize_mapping','w')
    h5s['mappings'] = mappings
    h5s['normal_features'] = normalize_all_features
    h5s.close()
    return home+'data/'+dataset+'/normalize_mapping',normalize_all_features,mappings
   
#-------------------------------------------------------------------------------
def data_selection(data,classes=[0,1,-1],combine=False,*args):              #which classes of data to keep
    if not isinstance(data,pd.DataFrame):
        cols = args
        data = pd.DataFrame(data,columns=cols)    
    ret_data = data.query('label in @classes')
    for i,label in enumerate(classes):
        ret_data['label'][ret_data['label']==label] = i
    
    if len(classes)>2 and combine==True:
        ret_data['label'] = ret_data['label'].where(ret_data['label']==0,1)
    return ret_data
        
#------------------------------------------------------------------------------                  
 
def TSNEPlot(data,class_labels,param_map):##param_map like 1:('r','o','80')
    data1 = data.reset_index()
    tsne = TSNE(n_components=3,random_state=91)
    feature_reduced = tsne.fit_transform(data1.drop(['label','index'],axis=1))
    fig = plt.figure(figsize=(18,15))
    ax = fig.gca(projection='3d')
    for i in class_labels:
        c,marker,size = param_map[i]
        print(c,marker,size)
        index = data1.query('label == @i').index.values
        ax.scatter(feature_reduced[index,0],feature_reduced[index,1],feature_reduced[index,2],c=c,marker=marker,s=size)
    plt.show()
    return None

#-------------------------------------------------------------------------------
def simulate(data,label,**argkw):
    data = data[data['label']==label]
    encoder = AS.dataset_simulator(**argkw)
    encoder.fit(np.array(data.drop('label',axis=1)))
    sim_data = pd.DataFrame(encoder.transform(),columns=data.columns.drop('label'))
    sim_data['label'] = label
    return sim_data
          
#-----------------------------------------------------------------------------
def subset_control(data,num):      ## randomly select a subset of certain ratio positive vs control samples 
    pos_index = data.query('label!=0').index.values
    control_sub_index = np.random.permutation(data.query('label==0').index.values)[:len(pos_index)*num]
    select_index = np.concatenate((pos_index,control_sub_index))
    return data.loc[select_index,:]
#------------------------------------------------------------------------------
def sparse_autoencoder_score(ae_params):
    global encoders
    sparse_ae = sac.sparse_autoencoder(**ae_params)
    score = commons.cross_validate_score(sparse_ae,reduced_train_x,train_label,ae_sample_weights_train)
    encoders.extend([sparse_ae])
    if math.isnan(score) or math.isfinite(score):
        score = np.Infinity
    return {'loss':-score,'status':STATUS_OK}
#-----------------------------------------------------------------
def selected_feature_analysis(features,X,y):
    stats = []
    pos = X[y==1]
    neg = X[y==0]
    for feature in features:
        #test = TTest.FeatureTTest(feature)
        test = WilcoxonRankSums.Ranksums(feature)
        stats.extend([test.fit(pos,neg)])
    return pd.DataFrame(stats)
#--------------------------------------------------------------------
def method_params(methods=['random_forest','xgboost','logistic_regression','linear_SVC']):
    params={}
    feature_num = train_x.shape[1]
    #class_weight = {0:1,1:30}
    class_weight = None
    l_param={'C':9,'penalty':'l1'}
    rf_param = {'n_estimators':1000,'max_depth':10,'min_samples_split':14 ,'min_samples_leaf':1, 'n_jobs':-1 }
    svc_param = {'C':2,'dual':False,'penalty':'l1'}
    xgb_param = {'learning_rate':0.1,'max_depth':10,'n_estimators':1500,'reg_lambda':40,'gamma':1,'n_jobs':-1}
    mutual_information_param = {'k':100}
    fisher_param = {'k':100}
    if 'logistic_regression' in methods:
        params['logistic_regression'] = l_param
    if 'random_forest' in methods:
        params['random_forest'] = rf_param
    if 'linear_SVC' in methods:
        params['linear_SVC'] = svc_param
    if 'xgboost' in methods:
        params['xgboost'] = xgb_param
    if 'mutual_information' in methods:
        params['mutual_information'] = mutual_information_param
    if 'fisher_score' in methods:
        params['fisher_score'] = fisher_param   
    return params

In [3]:
dataset = 'AD_CpG'
type_name = commons.type_name  ## amyloid, cerad, tangles
with_cell_type = commons.with_cell_type ## with or without
dataset = dataset+'/'+type_name+with_cell_type
log_dir = home+'logs/'
logger = Logger.Logger(log_dir,False).get_logger()
with pd.HDFStore(home+'data/'+dataset+'/all_features','r') as h5s:
    all_data = h5s['all_features']
all_data['beta_sign'] = all_data['label']
#all_data['coordinate'] = all_data['coordinate'].astype('i8')
all_data.drop(['coordinate','chr'],axis=1,inplace=True)
all_data['dist_to_nearest_tss'] = all_data['dist_to_nearest_tss'].astype('i8')
all_data = data_selection(all_data,classes=[0,1,-1],combine=True)
#all_data = data_selection(all_data,classes=[0,1,-1],combine=True)
all_features = all_data
#all_features = subset_control(all_data,30)
#all_features = all_data.query('beta_sign>0') ##only for hypermethylated sites in RICHS dataset, for AD dataset, hyper/hypo status can't be determined from beta
#logger.info('only keep heypermethylated sites')

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
dataset

'AD_CpG/braakwith'

In [5]:
if type_name == 'cerad':
    type_weight_factor = 0.23
elif type_name == 'amyloid':
    type_weight_factor = 0.3
elif type_name == 'cogdec':
    type_weight_factor = 0.4
elif type_name == 'gpath':
    type_weight_factor = 0.3
elif type_name == 'braak':
    type_weight_factor = 0.3
elif type_name == 'tangles':
    type_weight_factor = 0.3
else:
    type_weight_factor = 0.3

In [6]:
#split train test data and scaling on train data
scaler_type='MinMax'
all_features.drop(['id','winid','beta','beta_sign'],axis=1,inplace=True)
train_x,train_label,test_x,test_label,_ = commons.train_test_split(all_features,scaler=scaler_type)
train_x.reset_index(drop=True,inplace=True)
train_label.reset_index(drop=True,inplace=True)
test_x.reset_index(drop=True,inplace=True)
test_label.reset_index(drop=True,inplace=True)

In [7]:
all_features

,pvalue,label,A549,Astrocy,Colon_OC,Endometrium_OC,Frontal_cortex_OC,Gliobla,GM12878,GM12891,...,NCFF795DNO_WGBS_counts,NCFF801OHX_WGBS_counts,NCFF811QOG_WGBS_counts,NCFF831OYO_WGBS_counts,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,dist_to_nearest_tss
0,0.536040,0.0,3.0,2.0,5.0,6.0,1.0,1.0,3.0,4.0,...,1.0,70.0,12.0,10.0,6.0,4.0,3.0,14.0,6.0,2396
1,0.987950,0.0,3.0,4.0,4.0,4.0,2.0,10.0,11.0,19.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,353
2,0.515107,0.0,2.0,2.0,10.0,7.0,1.0,4.0,5.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,384
3,0.515261,0.0,9.0,9.0,9.0,9.0,1.0,12.0,13.0,32.0,...,3.0,66.0,21.0,23.0,10.0,1.0,9.0,10.0,1.0,13583
4,0.823404,0.0,2.0,5.0,5.0,3.0,4.0,8.0,6.0,6.0,...,3.0,48.0,7.0,15.0,13.0,12.0,21.0,9.0,6.0,21736
5,0.527860,0.0,5.0,7.0,13.0,11.0,2.0,10.0,18.0,25.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,211
6,0.410954,0.0,5.0,7.0,13.0,11.0,2.0,10.0,18.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,265
7,0.671217,0.0,1.0,5.0,1.0,3.0,0.0,6.0,6.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1219
8,0.731130,0.0,5.0,3.0,3.0,6.0,2.0,4.0,8.0,7.0,...,2.0,39.0,27.0,10.0,20.0,17.0,12.0,17.0,11.0,49667
9,0.425765,0.0,9.0,6.0,6.0,4.0,1.0,21.0,5.0,15.0,...,2.0,32.0,11.0,18.0,2.0,0.0,0.0,4.0,1.0,18134


In [8]:
sample_weights_train = commons.sample_weights(train_x,train_label,factor=1)
sample_weights_test = commons.sample_weights(test_x,test_label,factor=1)
weight_min_max_ratio = sample_weights_train.max()/sample_weights_train.min()
logger.info('weight max ratio: %f',weight_min_max_ratio)

In [9]:
weight_min_max_ratio

8694.158094569686

In [10]:
train_x.drop(['pvalue'],axis=1,inplace=True)
test_x.drop(['pvalue'],axis=1,inplace=True)

In [11]:
fs_sample_weights = np.power(sample_weights_train, type_weight_factor) 

In [12]:
fs_sample_weights.max()/fs_sample_weights.min()

37.64358815012941

In [13]:
methods = ['random_forest','xgboost','logistic_regression','linear_SVC']
all_intersect = False
fs_params = method_params(methods)
fs = FS.FeatureSelection(class_num=2,methods=methods,all_intersect=all_intersect,**fs_params)
logger.info('Feature selection methods are: '+str(methods))
logger.info('All intersected features: '+str(all_intersect))
fs.fit(sample_weight=fs_sample_weights)
selected_features = fs.transform(train_x,train_label)
logger.info('selected features number is: %d\n',selected_features.shape[0])
logger.info(selected_features)
reduced_train_x = train_x[selected_features['feature']]
reduced_test_x = test_x[selected_features['feature']]
total_x = pd.concat([reduced_train_x,reduced_test_x],ignore_index=True)
total_label = pd.concat([train_label,test_label],ignore_index=True)
total_weights = pd.concat([sample_weights_train,sample_weights_test],ignore_index=True)

feature_diff_stats = selected_feature_analysis(selected_features['feature'],total_x,total_label)
feature_diff_stats = pd.merge(feature_diff_stats,selected_features,on='feature')
selected_features_100 = feature_diff_stats if len(feature_diff_stats) <=50 else feature_diff_stats.sort_values(['n','pvalue'],ascending=[False,True])[:60]
reduced_train_x = train_x[selected_features_100['feature']]
reduced_test_x = test_x[selected_features_100['feature']]
total_x = pd.concat([reduced_train_x,reduced_test_x],ignore_index=True)
total_label = pd.concat([train_label,test_label],ignore_index=True)
total_weights = pd.concat([sample_weights_train,sample_weights_test],ignore_index=True)

[[0. 0. 0. ... 0. 0. 0.]]


In [14]:
selected_features_100

,diff(pos-neg),feature,pvalue,stats,n
327,0.034585,Penis_Foreskin_Fibroblast_Primary_Cells-H3K36me3,1.273059e-07,5.282683,4.0
334,0.032203,Skeletal_Muscle-H3K4me1,4.644749e-06,4.580225,4.0
333,0.039454,Dnd41-H4K20me1,9.063674e-06,4.438384,4.0
336,0.067274,Penis_Foreskin_Keratinocyte_Primary_Cells-H3K4me1,2.514063e-05,4.213533,4.0
331,0.112865,genocanyon_score,4.173064e-05,4.097683,4.0
330,0.054581,Spleen-H3K4me1,5.232399e-05,4.044996,4.0
338,0.034248,Rectal_Mucosa-H3K4me1,6.228165e-05,4.003995,4.0
337,-0.016807,Right_Atrium-H3K9me3,3.058025e-03,-2.961844,4.0
329,-0.015982,GM12878-EZH2,1.023607e-02,-2.567751,4.0
332,0.062983,NCFF692JTJ_WGBS_counts,1.429415e-02,2.449785,4.0


In [7]:
selected_features_100 = pd.read_csv(home+'data/'+dataset+'/feature_stats.csv')

In [8]:
_,_,_,_,scaler = commons.train_test_split(all_features[['label','pvalue']+list(selected_features_100['feature'])],scaler=scaler_type)
joblib.dump(scaler,home+'data/'+dataset+'/scaler.pkl')
print('Data scaler type is: %s'%scaler_type)

Data scaler type is: MinMax


In [33]:
selected_features_100

,Unnamed: 0,diff(pos-neg),feature,pvalue,stats,n
0,346,0.034585,Penis_Foreskin_Fibroblast_Primary_Cells-H3K36me3,1.273059e-07,5.282683,4.0
1,342,0.032203,Skeletal_Muscle-H3K4me1,4.644749e-06,4.580225,4.0
2,344,0.039454,Dnd41-H4K20me1,9.063674e-06,4.438384,4.0
3,348,0.067274,Penis_Foreskin_Keratinocyte_Primary_Cells-H3K4me1,2.514063e-05,4.213533,4.0
4,340,0.112865,genocanyon_score,4.173064e-05,4.097683,4.0
5,345,0.054581,Spleen-H3K4me1,5.232399e-05,4.044996,4.0
6,349,0.034248,Rectal_Mucosa-H3K4me1,6.228165e-05,4.003995,4.0
7,347,0.062983,NCFF692JTJ_WGBS_counts,1.429415e-02,2.449785,4.0
8,339,0.016642,Psoas_Muscle-H3K27ac,1.883180e-02,2.348844,4.0
9,343,0.011695,MCF-7-PML,2.524536e-02,2.237627,4.0


In [14]:
selected_features_100.to_csv(home+'data/'+dataset+'/feature_stats.csv')

In [ ]:
#TSNEPlot(plot_data,class_labels=[0,1,2],param_map={0:('g','^',20),1:('b','*',20),2:('r','o',20)})
plot_data = reduced_train_x.copy()
plot_data['label'] = train_label
TSNEPlot(plot_data,class_labels=[0,1],param_map={0:('k','^',20),1:('r','*',20)})

In [15]:
with pd.HDFStore(home+'data/'+dataset+'/selected_features','w') as h5s:
    h5s['train_x'] = reduced_train_x
    h5s['train_label'] = train_label
    h5s['test_x'] = reduced_test_x
    h5s['test_label'] = test_label
    h5s['sample_weights_train'] = sample_weights_train
    h5s['sample_weights_test'] = sample_weights_test